In [1]:
from crewai import LLM

llm = LLM(model="gpt-5-mini")

In [2]:
import yaml

with open('config/planner_agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/planner_tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

In [3]:
from pydantic import BaseModel
from typing import Optional

class Tweet(BaseModel):
    """Represents an individual tweet in a thread"""
    content: str
    is_hook: bool = False 
    media_urls: Optional[list[str]] = []

class Thread(BaseModel):
    """Represents a Twitter thread"""
    topic: str 
    tweets: list[Tweet]


In [4]:
from pydantic import BaseModel
from typing import Optional

class LinkedInPost(BaseModel):
    """Represents a LinkedIn post"""
    content: str  # The main content of the post
    media_url: str # Main image URL for the post


In [5]:
from crewai_tools import DirectoryReadTool, FileReadTool 

all_tools = [DirectoryReadTool(), FileReadTool()]

In [6]:
from crewai import Agent, Task

draft_analyzer = Agent(config=agents_config['draft_analyzer'],
                       tools=all_tools,
                       llm=llm)

analyze_draft = Task(config=tasks_config['analyze_draft'],
                     agent=draft_analyzer
                     )

In [7]:
from crewai import Agent, Task

twitter_thread_planner = Agent(config=agents_config['twitter_thread_planner'],
                               tools=all_tools,
                               llm=llm)

create_twitter_thread_plan = Task(config=tasks_config['create_twitter_thread_plan'],
                                  agent=twitter_thread_planner,
                                  output_pydantic=Thread)

In [8]:
from crewai import Agent, Task

linkedin_post_planner = Agent(config=agents_config['linkedin_post_planner'],
                              tools=all_tools,
                              llm=llm)

create_linkedin_post_plan = Task(config=tasks_config['create_linkedin_post_plan'],
                                 agent=linkedin_post_planner,
                                 output_pydantic=LinkedInPost)


In [9]:
from pydantic import BaseModel
from pathlib import Path

class ContentPlanningState(BaseModel):
    """
    State for the content planning flow
    """
    # URL of the blog to scrape
    blog_post_url: str = "https://blog.dailydoseofds.com/p/5-chunking-strategies-for-rag"
    
    # Path where the scraped content will be stored
    draft_path: Path = "assets/"
    
    # Determines whether to create a Twitter or LinkedIn post 
    post_type: str = "twitter"  
    
    # Example Twitter threads for style reference
    path_to_example_threads: str = "assets/example_threads.txt" 
    
    # Example LinkedIn posts for reference
    path_to_example_linkedin: str = "assets/example_linkedin.txt"

In [ ]:
from firecrawl import FirecrawlApp
import os
import uuid
import json
from crewai.flow import Flow, start,listen,router,or_
from typing import Optional


class CreateContentPlanningFlow(Flow[ContentPlanningState]):

    @start()
    def scrape_blog_post(self):
        print(f"# Fetching draft from: {self.state.blog_post_url}")

        # Initialize FireCrawl
        app = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
        
        # Scrape the blog post in Markdown and HTML format
        scrape_result = app.scrape(self.state.blog_post_url,
                                       formats= ['markdown', 'html'])

        # Extract the title (fallback to a UUID if not found)
        try:
            title = scrape_result['metadata']['title']
        except Exception:
            title = str(uuid.uuid4())

        # Store the scraped content as a markdown file
        self.state.draft_path = f'assets/{title}.md'
        with open(self.state.draft_path, 'w') as f:
            f.write(scrape_result.markdown)

        return self.state
    
    @router(scrape_blog_post)
    def select_platform(self):
        if self.state.post_type == "twitter":
            return "twitter"
        elif self.state.post_type == "linkedin":
            return "linkedin"
    
    @listen("twitter")
    def twitter_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")
    
        # Execute the Twitter Planning Crew
        result = twitter_planning_crew.kickoff(inputs={
            'draft_path': self.state.draft_path, 
            'path_to_example_threads': self.state.path_to_example_threads
        })
    
        print(f"# Planned content for {self.state.draft_path}:")
    
        # Print each tweet in the generated thread
        for i, tweet in enumerate(result.pydantic.tweets):
            print(f"Tweet {i+1}: {tweet.content}")
            print(f"Media URLs: {tweet.media_urls}")
            print("-" * 100)
    
        return result
    
    @listen("linkedin")
    def linkedin_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")
    
        # Execute the LinkedIn Planning Crew
        result = linkedin_planning_crew.kickoff(inputs={
            'draft_path': self.state.draft_path, 
            'path_to_example_linkedin': self.state.path_to_example_linkedin
        })
    
        print(f"# Planned content for {self.state.draft_path}:")
        print(f"{result.pydantic.content}")
    
        return result
    
    @listen(or_(twitter_draft, linkedin_draft))
    def save_plan(self, plan):
        with open(f'output/draft.json', 'w') as f:
            json.dump(plan.pydantic.model_dump(), f, indent=2)

In [ ]:
# corrected_content_planning_flow.py

from pydantic import BaseModel
from pathlib import Path
import os
import uuid
import json
from typing import Optional

# CrewAI imports
from crewai.flow import Flow, start, listen, router, or_

# Firecrawl (third-party)
from firecrawl import FirecrawlApp

# ---------- State model ----------
class ContentPlanningState(BaseModel):
    blog_post_url: str
    # where to save the scraped markdown file (string ok)
    draft_path: Optional[str] = None
    # type of post: "twitter" or "linkedin"
    post_type: Optional[str] = None
    path_to_example_threads: Optional[str] = None
    path_to_example_linkedin: Optional[str] = None

# ---------- Helper: robust call to Firecrawl ----------
def call_firecrawl_scrape(app, url, **params):
    """
    Try a few likely method names on the Firecrawl object and return the result.
    """
    candidate_names = ["scrape_url", "scrape", "crawl", "run", "fetch", "fetch_url"]
    for name in candidate_names:
        if hasattr(app, name):
            method = getattr(app, name)
            # call synchronous methods (adapt if any are async)
            try:
                # If callable, try to call
                return method(url, **params)
            except TypeError:
                # maybe signature uses params as a single dict or different arg name
                try:
                    return method(url, params)
                except Exception:
                    # last fallback: try without params
                    try:
                        return method(url)
                    except Exception:
                        pass
    # If nothing worked, raise a helpful error
    raise AttributeError(
        f"No supported scrape method found on Firecrawl object. Tried: {candidate_names}"
    )

# ---------- Flow ----------
class CreateContentPlanningFlow(Flow[ContentPlanningState]):

    @start()
    def scrape_blog_post(self):
        print(f"# Fetching draft from: {self.state.blog_post_url}")

        api_key = os.getenv("FIRECRAWL_API_KEY")
        if not api_key:
            raise RuntimeError("FIRECRAWL_API_KEY environment variable not set")

        # Ensure directories exist
        assets_dir = Path("assets")
        assets_dir.mkdir(parents=True, exist_ok=True)
        output_dir = Path("output")
        output_dir.mkdir(parents=True, exist_ok=True)

        # Initialize FireCrawl app
        app = FirecrawlApp(api_key=api_key)

        # Defensive: attempt to call the right method on the Firecrawl object
        try:
            scrape_result = call_firecrawl_scrape(app, self.state.blog_post_url,
                                                 params={"formats": ["markdown", "html"]})
        except AttributeError as e:
            # Optional: print available methods to debug
            available = sorted([m for m in dir(app) if not m.startswith("_")])
            raise RuntimeError(
                "Could not find a supported scrape method on FirecrawlApp. "
                f"Available public attributes/methods: {available}"
            ) from e

        # normalize result access (handle dict-like or object-like responses)
        markdown_text = None
        try:
            # prefer scrape_result['markdown']
            markdown_text = scrape_result.get("markdown") if isinstance(scrape_result, dict) else getattr(scrape_result, "markdown", None)
        except Exception:
            markdown_text = None

        # fallback: if markdown not present, try to convert from 'html' if available
        if not markdown_text:
            html_text = None
            try:
                html_text = scrape_result.get("html") if isinstance(scrape_result, dict) else getattr(scrape_result, "html", None)
            except Exception:
                html_text = None

            if html_text:
                # simple fallback: wrap html into markdown-ish form; for robust conversion use html2text
                try:
                    import html2text
                    markdown_text = html2text.html2text(html_text)
                except Exception:
                    # last resort: store raw html
                    markdown_text = f"<!-- original html fallback -->\n\n{html_text}"

        # Extract title (fallback to uuid)
        title = None
        try:
            if isinstance(scrape_result, dict):
                metadata = scrape_result.get("metadata", {}) or {}
                title = metadata.get("title")
            else:
                metadata = getattr(scrape_result, "metadata", None)
                if metadata:
                    title = metadata.get("title") if isinstance(metadata, dict) else getattr(metadata, "title", None)
        except Exception:
            title = None

        if not title:
            title = str(uuid.uuid4())

        # save markdown to assets/<title>.md
        filename = assets_dir / f"{title}.md"
        if markdown_text is None:
            markdown_text = f"# {title}\n\n(No markdown content extracted.)"

        with open(filename, "w", encoding="utf-8") as f:
            f.write(markdown_text)

        # update state
        self.state.draft_path = str(filename)

        return self.state

    @router(scrape_blog_post)
    def select_platform(self):
        # default to twitter if not set
        if not self.state.post_type:
            self.state.post_type = "twitter"
        if self.state.post_type.lower() == "twitter":
            return "twitter"
        elif self.state.post_type.lower() == "linkedin":
            return "linkedin"
        else:
            # fallback to twitter
            return "twitter"

    @listen("twitter")
    def twitter_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")

        # NOTE: ensure twitter_planning_crew is defined elsewhere in your notebook/module
        if "twitter_planning_crew" not in globals():
            raise RuntimeError("twitter_planning_crew is not defined. Define it before running the flow.")

        result = twitter_planning_crew.kickoff(inputs={
            "draft_path": self.state.draft_path,
            "path_to_example_threads": self.state.path_to_example_threads
        })

        print(f"# Planned content for {self.state.draft_path}:")
        # safe printing depending on result shape
        try:
            tweets = result.pydantic.tweets
            for i, tweet in enumerate(tweets):
                print(f"Tweet {i+1}: {getattr(tweet, 'content', tweet)}")
                print(f"Media URLs: {getattr(tweet, 'media_urls', '[]')}")
                print("-" * 80)
        except Exception:
            print("Result (unstructured):", result)

        return result

    @listen("linkedin")
    def linkedin_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")

        if "linkedin_planning_crew" not in globals():
            raise RuntimeError("linkedin_planning_crew is not defined. Define it before running the flow.")

        result = linkedin_planning_crew.kickoff(inputs={
            "draft_path": self.state.draft_path,
            "path_to_example_linkedin": self.state.path_to_example_linkedin
        })

        print(f"# Planned content for {self.state.draft_path}:")
        try:
            print(result.pydantic.content)
        except Exception:
            print("Result (unstructured):", result)

        return result

    @listen(or_(twitter_draft, linkedin_draft))
    def save_plan(self, plan):
        # Ensure output dir
        Path("output").mkdir(parents=True, exist_ok=True)
        out_file = Path("output/draft.json")
        with open(out_file, "w", encoding="utf-8") as f:
            # try to serialise pydantic result safely
            try:
                json.dump(plan.pydantic.model_dump(), f, indent=2)
            except Exception:
                # fallback: stringify
                json.dump({"raw": str(plan)}, f, indent=2)
        print(f"Saved plan to {out_file}")
        return plan


In [ ]:
from crewai import Crew


twitter_planning_crew = Crew(

    agents=[draft_analyzer, twitter_thread_planner],

    tasks=[analyze_draft, create_twitter_thread_plan],

    verbose=True

)


linkedin_planning_crew = Crew(

    agents=[draft_analyzer, linkedin_post_planner],

    tasks=[analyze_draft, create_linkedin_post_plan],

    verbose=True
    )

In [ ]:
2+3

5

In [17]:
flow = CreateContentPlanningFlow()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: CreateContentPlanningFlow                                                                                │
│  ID: 5dbe9e03-b26b-4cf1-8f76-0c3f71279af7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
flow.plot()

'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\crewai_flow_s6wn5ukn\\crewai_flow.html'

In [ ]:
# Inspect the FirecrawlApp instance to find where the scraping method actually lives.
# Run AFTER you create `app = FirecrawlApp(api_key=...)` in the same cell or earlier cell.

def print_firecrawl_structure(obj, depth=1, prefix=""):
    try:
        attrs = sorted([a for a in dir(obj) if not a.startswith("_")])
    except Exception as e:
        print("Could not list dir():", e)
        return
    print(f"{prefix}{obj.__class__} -> public attrs ({len(attrs)}):")
    for a in attrs:
        print(prefix + "  -", a)
    if depth > 0:
        # inspect nested objects that look interesting (common names)
        for candidate in ("client", "firecrawl", "api", "backend"):
            if hasattr(obj, candidate):
                nested = getattr(obj, candidate)
                print("\n" + prefix + f"Nested '{candidate}' -> {nested.__class__}")
                try:
                    nested_attrs = sorted([a for a in dir(nested) if not a.startswith("_")])
                    for a in nested_attrs:
                        print(prefix + "   *", a)
                except Exception as e:
                    print(prefix + "   (couldn't list nested dir)", e)

# Example: create app and inspect it
app = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
print_firecrawl_structure(app, depth=1)


<class 'firecrawl.client.Firecrawl'> -> public attrs (28):

- active_crawls

- api_key

- api_url

- batch_scrape

...

- start_crawl

- start_extract

- v1

- v2

- watcher

In [18]:
# In a notebook cell
result = await flow.kickoff_async()


Flow started with ID: 5dbe9e03-b26b-4cf1-8f76-0c3f71279af7


Flow started with ID: 5dbe9e03-b26b-4cf1-8f76-0c3f71279af7
# Fetching draft from: https://blog.dailydoseofds.com/p/5-chunking-strategies-for-rag


Output()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 32c1fdbb-4907-41ed-b158-69049f354232                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Draft Analyzer                                                                                          │
│                                                                                                                 │
│  Task: Analyze the markdown file at assets/eabc41eb-71c0-42af-8773-cd706008b584.md to create a                  │
│  developer-focused                                                                                              │
│  technical overview                                                                                             │
│                                                                                                                 │
│  1. Map out the core idea that the blog discusses                                                               │
│                                                                                                                 │
│  2. Identify key sections and what each section is about                                                        │
│                                                                                                                 │
│  3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)                  │
│                                                                                                                 │
│  4. You must associate these identified image urls to their                                                     │
│     correspoinding sections, so that we can use them with the tweets as media pieces                            │
│                                                                                                                 │
│  Focus on details that are important for a comprehensive understanding of                                       │
│  the blog.                                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Tweet 3: Why chunk? Embedding input limits, retrieval signal, and what you can pass into the LLM. Bad chunks = diluted embeddings = worse answers. So chunking = critical preprocessing.
Media URLs: ['https://substackcdn.com/image/fetch/$s_!DpHM!,w_1456,c_limit,f_auto,q_auto:good,fl_lossy/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F6878b8fa-5e74-45a1-9a89-5aab92889126_2366x990.gif']
----------------------------------------------------------------------------------------------------
Tweet 4: Overview — the five strategies: Fixed-size, Semantic, Recursive, Document-structure, and LLM-based. Pick based on doc type, budget, and retrieval quality needs.
Media URLs: ['https://substackcdn.com/image/fetch/$s_!us2E!,w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F92c70184-ba0f-4877-9a55-e4add0e311ad_870x1116.gif']
--------------------------------------------------------------------------------

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: CreateContentPlanningFlow                                                                                │
│  ID: 5dbe9e03-b26b-4cf1-8f76-0c3f71279af7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

╭─────────────────────────────────────────── Tracing Preference Saved ────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing has been disabled.                                                                               │
│                                                                                                                 │
│  Your preference has been saved. Future Crew/Flow executions will not collect traces.                           │
│                                                                                                                 │
│  To enable tracing later, do any one of these:                                                                  │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [22]:
from IPython.display import Markdown,display
Markdown(display(result))

None

<IPython.core.display.Markdown object>

In [23]:
flow.state.post_type = "linkedin"
result1 = await flow.kickoff_async()

Flow started with ID: 5dbe9e03-b26b-4cf1-8f76-0c3f71279af7
# Fetching draft from: https://blog.dailydoseofds.com/p/5-chunking-strategies-for-rag


Output()

# Planning content for: assets/127ec84a-a428-44e5-8f90-079985f8e04b.md

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 52831f74-edf3-44d3-8734-2db1593e0dcf                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Draft Analyzer                                                                                          │
│                                                                                                                 │
│  Task: Analyze the markdown file at assets/127ec84a-a428-44e5-8f90-079985f8e04b.md to create a                  │
│  developer-focused                                                                                              │
│  technical overview                                                                                             │
│                                                                                                                 │
│  1. Map out the core idea that the blog discusses                                                               │
│                                                                                                                 │
│  2. Identify key sections and what each section is about                                                        │
│                                                                                                                 │
│  3. For each section, extract all URLs that appear inside image markdown syntax ![](image_url)                  │
│                                                                                                                 │
│  4. You must associate these identified image urls to their                                                     │
│     correspoinding sections, so that we can use them with the tweets as media pieces                            │
│                                                                                                                 │
│  Focus on details that are important for a comprehensive understanding of                                       │
│  the blog.                                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LinkedIn Post Planner                                                                                   │
│                                                                                                                 │
│  Thought: Thought: you should always think about what to do                                                     │
│                                                                                                                 │
│  Using Tool: List files in directory                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LinkedIn Post Planner                                                                                   │
│                                                                                                                 │
│  Thought: Thought: you should always think about what to do                                                     │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LinkedIn Post Planner                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "content": "5 Chunking Strategies For RAG\n\n5 Chunking Strategies For RAG — why chunking is the unsung      │
│  hero of Retrieval-Augmented Generation (RAG).\n\nWhy this matters\n- RAG pipelines rely on embeddings +        │
│  vector search to retrieve context for an LLM. Large documents must be split into chunks so they fit embedding  │
│  model token limits and return relevant, focused context to the LLM.\n- Poor chunking => noisy retrieval,       │
│  higher hallucination risk, worse customer outcomes, and wasted compute.\n\nThe core technical problem          │
│  (brief)\n- Embedding models have input-size/token limits. Documents larger than that must be partitioned.\n-   │
│  How you chunk affects retrieval precision: split too coarsely and you return irrelevant material; split too    │
│  finely and you lose semantic coherence.\n- Business impact: retrieval quality affects response accuracy, user  │
│  trust, support costs, and downstream KPIs (conversion, retention, risk).\n\nFive practical chunking            │
│  strategies (what they are, pros & cons)\n\n1) Fixed-size chunking\n- What: split by characters/words/tokens    │
│  into uniform segments; often add overlap.\n- Pros: trivial to implement, uniform batches for fast embedding,   │
│  predictable costs.\n- Cons: can slice semantic units mid-idea; retrieval may return fragments lacking          │
│  coherence.\n- When to use: short documents or when you need a fast, simple baseline.\n\n2) Semantic            │
│  chunking\n- What: segment into sentences/paragraphs, compute embeddings per segment, grow chunks while         │
│  adjacent segments are above a cosine-similarity threshold.\n- Pros: preserves natural language flow; richer    │
│  chunks => better retrieval relevance.\n- Cons: requires threshold tuning per document type; extra embedding    │
│  work for segments.\n- When to use: unstructured long text (blog posts, articles) where semantic coherence is   │
│  important.\n\n3) Recursive chunking\n- What: prefer natural separators (headings/paragraphs); if a partition   │
│  exceeds token limits, split it recursively until it fits.\n- Pros: preserves semantic cohesion and respects    │
│  model limits.\n- Cons: more implementation complexity and compute overhead.\n- When to use: large documents    │
│  with mixed-length sections (reports, whitepapers).\n\n4) Document-structure-based chunking\n- What: use        │
│  headings, sections, and explicit structure to set chunk boundaries.\n- Pros: aligns with author intent and     │
│  keeps logical sections intact.\n- Cons: fails when source lacks clear structure; sections may still exceed     │
│  token limits (combine with recursive splitting).\n- When to use: well-structured content (specs, manuals,      │
│  legal docs).\n\n5) LLM-based chunking\n- What: prompt an LLM to produce semantically isolated chunks (or       │
│  summarize/segment for chunk creation).\n- Pros: high semantic fidelity — the LLM understands context beyond    │
│  naive heuristics.\n- Cons: computationally expensive and subject to the LLM’s own context limits; consider     │
│  hierarchical or streaming approaches.\n- When to use: high-value corpora where semantic accuracy justifies     │
│  cost (legal, clinical, enterprise knowledge bases).\n\

Output()